### Transport Tutorial - Notebook 2

# Implement and solve a variation of *Dantzig's Transport Problem*

## Aim and scope of this tutorial

This tutorial takes you through the steps to solve a simple optimization model
using the **ixmp4** database management package and the **linopy** optimization package.

We use **Dantzig's transport problem**, which is used as a [tutorial for linopy](https://linopy.readthedocs.io/en/latest/transport-tutorial.html).
This problem solves for a least-cost shipping schedule that meets demand constraints in several markets (cities)
and supply constraints at factories.

For reference of the transport problem, see:
> Dantzig, G B, Chapter 3.3. In Linear Programming and Extensions.  
> Princeton University Press, Princeton, New Jersey, 1963.

## Tutorial outline

This tutorial consists of three Jupyter notebooks:

0. Set up an **ixmp4.Platform** to store the scenario input data and solution
1. Implement the **baseline version of the transport problem** and solve it
2. Create an **alternative scenario** and solve it 

<div class="alert alert-info">

This notebook requires that you created and solved the base transport problem as shown in [**Notebook 1**](1_transport-tutorial.ipynb).

</div>

## The platform as a connection to the database

An [**ixmp4.Platform**](https://docs.ece.iiasa.ac.at/projects/ixmp4/en/latest/devs/ixmp4.core/platform.html#ixmp4.core.platform.Platform)
is the connection to a database instance that can hold scenario data and relevant additional information.

In [ ]:
import ixmp4
import pandas as pd

In [ ]:
platform = ixmp4.Platform("transport-tutorial")

As a first step, we list all scenarios (aka [**ixmp4.Run**](https://docs.ece.iiasa.ac.at/projects/ixmp4/en/latest/devs/ixmp4.core/run.html#ixmp4.core.run.Run) instances)
available in the platform connected to the database instance.

In [ ]:
platform.runs.tabulate()

<div class="alert alert-info">

If you are running this tutorial for the first time, you will only see the *standard* scenario of the *transport problem*.  
If you have already done some experimentation in the `transport-tutorial` platform, you will see more scenarios.

</div>

## Retrieve a run from the platform

As a first step in this notebook, you can load the **ixmp4.Run** that you created in Notebook 1.

In [ ]:
run = platform.runs.get(model="transport problem", scenario="standard")

You can check that this is indeed the version shown above by comparing the *version number*.

In [ ]:
run.version

You can check that the solution from Notebook 1 was indeed saved by inspecting an **ixmp4.Variable**.  
The `data` of any optimization object is stored internally as a dictionary.

In [ ]:
run.optimization.variables.get("z").data

For instances of a **Variable** or a **Parameter** that have multiple dimensions,
you can use **pandas** for filtering and inspecting the data.

The following cell shows the data for a using the distance parameter *d* and cast the `data` to a **pandas.DataFrame**.

In [ ]:
d = run.optimization.parameters.get("d")
distance = pd.DataFrame(d.data)

In [ ]:
distance[distance["i"] == "seattle"]

You can also combine filters for more elaborate queries.

In [ ]:
distance.loc[(distance["i"] == "seattle") & (distance["j"].isin(["chicago", "topeka"]))]

Please note that **pandas** recommends to use [advanced indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced-advanced-hierarchical) if you find yourself using more than three conditionals in `.loc[]`.

## Make a clone of the baseline scenario

For illustration of a typical scenario analysis workflow, we create a copy of the standard transportation problem
and make two modifications:
- Add a new demand location *detroit* and add a demand-level and freight costs to that city.
- Reduce the demand level in *chicago* to ensure a feasible problem, because the total production capacity does not allow much slack for increased production.

The next cell creates a copy (`clone()`) of the standard transportation problem.

In [ ]:
run_detroit = run.clone(scenario="detroit")

Because the cell above did not specify a *model name*, the clone kept the original model name.

In [ ]:
run_detroit.model.name

We now modify the structure and parameters of the *detroit* scenario.
First, we reduce the demand in *chicago* to ensure a feasible problem..

In [ ]:
run_detroit.optimization.parameters.get("b").add(
    {"j": ["chicago"], "values": [200], "units": ["cases"]}
)

Now, we add a new market *detroit* and add all relevant parameters.

In [ ]:
run_detroit.optimization.indexsets.get("j").add("detroit")
run_detroit.optimization.parameters.get("b").add(
    {"j": ["detroit"], "values": [150], "units": ["cases"]}
)
run_detroit.optimization.parameters.get("d").add(
    {
        "i": ["seattle", "san-diego"],
        "j": ["detroit", "detroit"],
        "values": [1.7, 1.9],
        "units": ["km", "km"],
    }
)

### Solve the new scenario

Now, we create a **linopy** model, solve it, and read the solution to store it
in the respective **Variable** and **Equation** objects of the **ixmp4.Run**.

In [ ]:
from tutorial.transport.dantzig_model_linopy import (
    create_dantzig_model,
    read_dantzig_solution,
)


linopy_model_detroit = create_dantzig_model(run=run_detroit)
linopy_model_detroit.solve("highs")
read_dantzig_solution(model=linopy_model_detroit, run=run_detroit)

## Setting a default version of the scenario

As in Notebook 1, we set the *detroit* scenario as the default version so that we can easily retrieve it later.

In [ ]:
run_detroit.set_as_default()

## Display and analyze the results

We can now compare the *standard* instance of the *transport problem* and the *detroit* scenario.

The next two cells show the objective values of the two runs.

In [ ]:
run.optimization.variables.get("z").levels

In [ ]:
run_detroit.optimization.variables.get("z").levels

You see that the *detroit* scenario has higher total cost.

The next two cells compare the optimal assignment of shipments from plants to markets in the two runs.

In [ ]:
pd.DataFrame(run.optimization.variables.get("x").data)

In [ ]:
pd.DataFrame(run_detroit.optimization.variables.get("x").data)

In [ ]:
run_detroit.optimization.equations.get("demand").data

## Scenario version management

As above, you can now list all scenarios available in the database instance to which the **ixmp4.Platform** is connected.

You should now see (at least) two scenarios.

In [ ]:
platform.runs.tabulate()

When developing scenarios, we often make minor variations to numerous scenarios, and changing the *scenario name* every time is not practical.

The **ixmp4** package provides a more efficient approach to scenario version management: Every time that you *create* a new run or *clone* an existing run and the *model-scenario-name* combination already exists, a new *version* will be created.

You can get a specific *version* by including the version number in the call to get an **ixmp4.Run** like

```python
platforms.runs.get(model, scenario, version)
```

You also can assign one of the versions as *default*, so that this run will be retrieved when you call

```python
platforms.runs.get(model, scenario)
```

without a version number.

All versions will be kept in the database and are accessible via the **ixmp4.Platform**.

If you have run this tutorial multiple times, you can tabulate all runs - including the non-default versions - as illustrated below.

In [ ]:
platform.runs.tabulate(default_only=False)

## Cleaning up after doing this tutorial

You can use the **ixmp4** Command Line Interface (CLI) to remove the database related to this tutorial.

Run the following in the command-line:
```
ixmp4 platforms delete transport-tutorial
```

The prompt will ask whether you also want to delete the SQLite database file from your computer.